In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import datetime
from bs4 import BeautifulSoup
import urllib.parse as urlparse


def get_request_query(url, operation, params, serviceKey):
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

In [7]:
mykey = "any0XDsU8nQxEH9Y7fOm9HUGLhh06m6rT88qKDZ6YYpBQ%2BuIUYImLHJcGHttmoWXiQ1C%2FxdsoXwolbw2Do9kGQ%3D%3D"
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
operation = 'getRestDeInfo'
year = '2021'
month = '09'

params = {'solYear':year, 'solMonth':month}

request_query = get_request_query(url, operation, params, mykey)
print(request_query)

http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear=2021&solMonth=09&serviceKey=any0XDsU8nQxEH9Y7fOm9HUGLhh06m6rT88qKDZ6YYpBQ%2BuIUYImLHJcGHttmoWXiQ1C%2FxdsoXwolbw2Do9kGQ%3D%3D


In [ ]:
# 2016~2021년 4월까지 공휴일

In [31]:
df = pd.DataFrame()

for year in [2016, 2017, 2018, 2019, 2020, 2021]:
    for month in range(1,13):

        if month < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        params = {'solYear':year, 'solMonth':month}

        request_query = get_request_query(url, operation, params, mykey)

        res = requests.get(request_query)
        soup = BeautifulSoup(res.text, 'lxml')
        items = soup.find_all('item')

        for item in items:
            day = item.locdate.get_text()
            name = item.datename.get_text()
            data = {'휴일이름':name , '일자':day}
            df = df.append(data, ignore_index=True)
    
display(df.head())
display(df.tail())

,일자,휴일이름
0,20160101,신정
1,20160207,설날
2,20160208,설날
3,20160209,설날
4,20160210,대체공휴일


,일자,휴일이름
101,20211003,개천절
102,20211004,대체공휴일
103,20211009,한글날
104,20211011,대체공휴일
105,20211225,기독탄신일


In [33]:
df['일자'] = pd.to_datetime(df['일자'], format = '%Y-%m-%d')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      106 non-null    datetime64[ns]
 1   휴일이름    106 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1.8+ KB


In [36]:
holiday = df.query('"2016-02-01"<= 일자 <= "2021-04-09"')
holiday

,일자,휴일이름
1,2016-02-07,설날
2,2016-02-08,설날
3,2016-02-09,설날
4,2016-02-10,대체공휴일
5,2016-03-01,삼일절
...,...,...
88,2021-01-01,1월1일
89,2021-02-11,설날
90,2021-02-12,설날
91,2021-02-13,설날


In [37]:
holiday.reset_index(drop=True)

,일자,휴일이름
0,2016-02-07,설날
1,2016-02-08,설날
2,2016-02-09,설날
3,2016-02-10,대체공휴일
4,2016-03-01,삼일절
...,...,...
87,2021-01-01,1월1일
88,2021-02-11,설날
89,2021-02-12,설날
90,2021-02-13,설날


In [38]:
# train 기간 분리

train_holiday = df.query('"2016-02-01"<= 일자 <= "2021-04-09"')
train_holiday

,일자,휴일이름
1,2016-02-07,설날
2,2016-02-08,설날
3,2016-02-09,설날
4,2016-02-10,대체공휴일
5,2016-03-01,삼일절
...,...,...
88,2021-01-01,1월1일
89,2021-02-11,설날
90,2021-02-12,설날
91,2021-02-13,설날


In [39]:
# train 기간 공휴일 데이터 저장

df = pd.DataFrame(train_holiday)
df.to_csv('train_holiday.csv', index=False)

In [40]:
# test 기간 분리

test_holiday = df.query('"2021-01-27"<= 일자 <= "2021-04-09"')
test_holiday

,일자,휴일이름
89,2021-02-11,설날
90,2021-02-12,설날
91,2021-02-13,설날
92,2021-03-01,삼일절


In [41]:
# test 기간 공휴일 데이터 저장

df = pd.DataFrame(test_holiday)
df.to_csv('test_holiday.csv', index=False)